## Workshop - Regularization

In this workshop, we are going to:

1. Tune an elastic-net regression 
2. Compare the following models:
    1. The null model
    2. The tuned elastic-net model
    3. The trimmed non-regularized model with standardized features
    4. The trimmed non-regularized model with non-standardized features
    
# Preliminaries

- Load any necessary packages and/or functions
- Load in and prepare the class data
- Create x and y with a label of `pct_d_rgdp`
- Create `x_train`, `x_test`, `y_train`, `y_test` with
    * training size of two-thirds
    * random state of 490
- Standardize the features
- Add constants

In [66]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn import linear_model as lm

In [67]:
df = pd.read_csv('class_data.csv')

In [68]:
df = df.drop(columns = ['fips','GeoName'])

In [69]:
df_prepped = df.drop(columns = ['urate_bin', 'year']).join([
    pd.get_dummies(df['urate_bin'], drop_first = True),
    pd.get_dummies(df.year, drop_first = True)    
])

In [70]:
y = df_prepped['pct_d_rgdp']
x = df_prepped.drop(columns = 'pct_d_rgdp')

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 2/3, random_state = 490)

x_train_std = x_train.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)
x_test_std  = x_test.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)

x_train_std = sm.add_constant(x_train_std)
x_test_std  = sm.add_constant(x_test_std)
x_train     = sm.add_constant(x_train)
x_test      = sm.add_constant(x_test)


Take a look at `lm.ElasticNet?` and 
```
fit = sm.OLS(y_train, x_train)
fit.fit_regularized?
```
Determine which coefficients are the same, but named differently.
Specifically, $\alpha$ and the weight on the different constraints (i.e. $||\beta||_2$ and $||\beta||_1$).

In [71]:
fit = sm.OLS(y_train, x_train)

In [85]:
fit_elasticnet = fit.fit_regularized(method = 'elastic_net', alpha = 1, L1_wt=1)
fit_elasticnet.params

const                0.000000e+00
pos_net_jobs         0.000000e+00
emp_estabs           0.000000e+00
estabs_entry_rate    1.913649e-01
estabs_exit_rate    -9.842778e-02
pop                 -2.365903e-07
pop_pct_black       -2.046507e-02
pop_pct_hisp         1.434198e-02
lfpr                 1.614383e-02
density              0.000000e+00
lower                0.000000e+00
similar              0.000000e+00
2003                 0.000000e+00
2004                 0.000000e+00
2005                 0.000000e+00
2006                 0.000000e+00
2007                 0.000000e+00
2008                 0.000000e+00
2009                 0.000000e+00
2010                 0.000000e+00
2011                 0.000000e+00
2012                 0.000000e+00
2013                 0.000000e+00
2014                 0.000000e+00
2015                 0.000000e+00
2016                 0.000000e+00
2017                 0.000000e+00
2018                 0.000000e+00
dtype: float64

Perform a 5-fold cross-validation grid search with a random state of 490. 
Identify the optimally tuned hyperparameters.
Use this grid:
```
param_grid = {'alpha': 10.**np.arange(-5, -1, 1), 
              'l1_ratio': np.arange(0, 1, 0.1)}
```
You will get a warning message about convergence.
We will discuss it after the workshop.
Think about why it occuring.

In [73]:
param_grid = [{'alpha': 10.**np.arange(-5, -1, 1), 'l1_ratio': np.arange(0, 1, 0.1)}]
cv_elasticnet = lm.ElasticNet(fit_intercept = False, normalize = False,
                    random_state = 490)

grid_search = GridSearchCV(cv_elasticnet, param_grid, cv = 5,
                         scoring = 'neg_root_mean_squared_error')
grid_search.fit(x_train_std, y_train)
alpha_best = grid_search.best_params_['alpha']
print(alpha_best)
ratio_best = grid_search.best_params_['l1_ratio']
print(ratio_best)

/Users/juli/opt/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1162767.3929822615, tolerance: 253.23442781744671
  model = cd_fast.enet_coordinate_descent(
/Users/juli/opt/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1130609.4852849494, tolerance: 246.96229539243058
  model = cd_fast.enet_coordinate_descent(
/Users/juli/opt/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1140144.213104668, tolerance: 248.68990538784342
  model = cd_fast.enet_coordinate_descent(
/Users/juli/opt/miniconda3/lib/python3.8/site-packages/sklearn/linea

0.01
0.0


/Users/juli/opt/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1449757.3812360945, tolerance: 315.5255958178445
  model = cd_fast.enet_coordinate_descent(


****
# Question

How many models did we just fit?

200 models(5 folders * 4 alphas * 10 ratios)

***
Using the tuned hyperparameters, fit your elastic net model with `statsmodels`

In [74]:
fit_elastic_tuned = sm.OLS(y_train, x_train_std).fit_regularized(L1_wt = ratio_best,alpha = alpha_best, method = "elastic_net")
fit_elastic_tuned.params

array([ 1.96364974,  0.56560128, -0.1799134 ,  0.87161969, -0.52587028,
       -0.10111834,  0.01932953,  0.31077548,  0.48842639, -0.00830354,
        0.60185639,  0.25471323, -0.05717825, -0.12751556,  0.01240067,
        0.38462281, -0.3780806 , -0.41076982, -0.59953967,  0.11248966,
       -0.14950141, -0.44118467, -0.02749793, -0.36985377, -0.29046667,
       -0.63804809, -0.2729354 , -0.12243805])

Using the selected features refit

- the non-regularized model with standardized features
- the non-regularized model with non-standardized features

In [75]:
beta = pd.Series(fit_elastic_tuned.params, index = x_train_std.columns)
beta.index[beta == 0]

Index([], dtype='object')

In [76]:
x_train_std_trim = x_train_std.loc[:, ~x_train_std.columns.isin(beta.index[beta == 0])]
x_test_std_trim = x_test_std.loc[:, ~x_test_std.columns.isin(beta.index[beta == 0])]

In [77]:
fit_std_final = sm.OLS(y_train, x_train_std_trim).fit()
fit_std_final.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
====================================================================
Model:              OLS              Adj. R-squared:     0.041      
Dependent Variable: pct_d_rgdp       AIC:                246976.3293
Date:               2021-02-26 19:14 BIC:                247212.3930
No. Observations:   33889            Log-Likelihood:     -1.2346e+05
Df Model:           27               F-statistic:        54.18      
Df Residuals:       33861            Prob (F-statistic): 2.24e-285  
R-squared:          0.041            Scale:              85.550     
--------------------------------------------------------------------
                      Coef.  Std.Err.    t    P>|t|   [0.025  0.975]
--------------------------------------------------------------------
const                 1.9833   0.0502 39.4735 0.0000  1.8848  2.0818
pos_net_jobs          0.5720   0.0546 10.4783 0.0000  0.4650  0.6791
emp_estabs           -0.1838   0.0544 -3.3801 0.0007 -0.2903 -0.0772
estabs_entry_rate     0.8761   0.0607 14.4365 0.0000  0.7572  0.9951
estabs_exit_rate     -0.5393   0.0583 -9.2538 0.0000 -0.6535 -0.4250
pop                  -0.1024   0.0568 -1.8034 0.0713 -0.2136  0.0089
pop_pct_black         0.0237   0.0573  0.4138 0.6790 -0.0886  0.1361
pop_pct_hisp          0.3156   0.0523  6.0394 0.0000  0.2132  0.4181
lfpr                  0.4893   0.0624  7.8430 0.0000  0.3671  0.6116
density              -0.0080   0.0540 -0.1478 0.8825 -0.1138  0.0978
lower                 0.6112   0.0684  8.9412 0.0000  0.4772  0.7452
similar               0.2618   0.0591  4.4314 0.0000  0.1460  0.3775
2003                 -0.0918   0.0697 -1.3165 0.1880 -0.2284  0.0449
2004                 -0.1632   0.0698 -2.3385 0.0194 -0.2999 -0.0264
2005                 -0.0191   0.0728 -0.2623 0.7931 -0.1618  0.1236
2006                  0.3570   0.0732  4.8782 0.0000  0.2135  0.5004
2007                 -0.4158   0.0697 -5.9662 0.0000 -0.5524 -0.2792
2008                 -0.4472   0.0697 -6.4138 0.0000 -0.5838 -0.3105
2009                 -0.6359   0.0707 -8.9950 0.0000 -0.7745 -0.4973
2010                  0.0809   0.0706  1.1470 0.2514 -0.0574  0.2193
2011                 -0.1847   0.0709 -2.6038 0.0092 -0.3237 -0.0457
2012                 -0.4797   0.0709 -6.7700 0.0000 -0.6186 -0.3408
2013                 -0.0619   0.0708 -0.8752 0.3815 -0.2007  0.0768
2014                 -0.4090   0.0718 -5.7003 0.0000 -0.5496 -0.2684
2015                 -0.3281   0.0708 -4.6337 0.0000 -0.4669 -0.1893
2016                 -0.6795   0.0712 -9.5455 0.0000 -0.8191 -0.5400
2017                 -0.3102   0.0711 -4.3622 0.0000 -0.4495 -0.1708
2018                 -0.1586   0.0713 -2.2239 0.0262 -0.2983 -0.0188
--------------------------------------------------------------------
Omnibus:            34608.272     Durbin-Watson:        1.994       
Prob(Omnibus):      0.000         Jarque-Bera (JB):     10681061.763
Skew:               4.484         Prob(JB):             0.000       
Kurtosis:           89.509        Condition No.:        6           
====================================================================

"""

In [78]:
x_train_trim = x_train.loc[:, ~x_train.columns.isin(beta.index[beta == 0])]
x_test_trim = x_test.loc[:, ~x_test.columns.isin(beta.index[beta == 0])]

In [79]:
fit_final = sm.OLS(y_train, x_train_trim).fit()
fit_final.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
====================================================================
Model:              OLS              Adj. R-squared:     0.041      
Dependent Variable: pct_d_rgdp       AIC:                246976.3293
Date:               2021-02-26 19:15 BIC:                247212.3930
No. Observations:   33889            Log-Likelihood:     -1.2346e+05
Df Model:           27               F-statistic:        54.18      
Df Residuals:       33861            Prob (F-statistic): 2.24e-285  
R-squared:          0.041            Scale:              85.550     
--------------------------------------------------------------------
                      Coef.  Std.Err.    t    P>|t|   [0.025  0.975]
--------------------------------------------------------------------
const                -2.1296   0.5863 -3.6321 0.0003 -3.2788 -0.9804
pos_net_jobs          1.1525   0.1100 10.4783 0.0000  0.9369  1.3681
emp_estabs           -0.0385   0.0114 -3.3801 0.0007 -0.0608 -0.0162
estabs_entry_rate     0.2894   0.0200 14.4365 0.0000  0.2501  0.3287
estabs_exit_rate     -0.2108   0.0228 -9.2538 0.0000 -0.2554 -0.1661
pop                  -0.0000   0.0000 -1.8034 0.0713 -0.0000  0.0000
pop_pct_black         0.0016   0.0039  0.4138 0.6790 -0.0061  0.0093
pop_pct_hisp          0.0241   0.0040  6.0394 0.0000  0.0163  0.0319
lfpr                  0.0439   0.0056  7.8430 0.0000  0.0329  0.0548
density              -0.0000   0.0000 -0.1478 0.8825 -0.0001  0.0001
lower                 1.2597   0.1409  8.9412 0.0000  0.9836  1.5358
similar               0.6765   0.1527  4.4314 0.0000  0.3773  0.9757
2003                 -0.3885   0.2951 -1.3165 0.1880 -0.9668  0.1899
2004                 -0.6965   0.2978 -2.3385 0.0194 -1.2803 -0.1127
2005                 -0.0808   0.3080 -0.2623 0.7931 -0.6846  0.5230
2006                  1.5046   0.3084  4.8782 0.0000  0.9001  2.1091
2007                 -1.7536   0.2939 -5.9662 0.0000 -2.3298 -1.1775
2008                 -1.9061   0.2972 -6.4138 0.0000 -2.4886 -1.3236
2009                 -2.6897   0.2990 -8.9950 0.0000 -3.2758 -2.1036
2010                  0.3421   0.2983  1.1470 0.2514 -0.2425  0.9268
2011                 -0.7776   0.2986 -2.6038 0.0092 -1.3630 -0.1923
2012                 -2.0476   0.3024 -6.7700 0.0000 -2.6404 -1.4548
2013                 -0.2639   0.3015 -0.8752 0.3815 -0.8548  0.3271
2014                 -1.7180   0.3014 -5.7003 0.0000 -2.3087 -1.1273
2015                 -1.4179   0.3060 -4.6337 0.0000 -2.0177 -0.8181
2016                 -2.8945   0.3032 -9.5455 0.0000 -3.4889 -2.3002
2017                 -1.3208   0.3028 -4.3622 0.0000 -1.9143 -0.7274
2018                 -0.6856   0.3083 -2.2239 0.0262 -1.2898 -0.0813
--------------------------------------------------------------------
Omnibus:            34608.272     Durbin-Watson:        1.994       
Prob(Omnibus):      0.000         Jarque-Bera (JB):     10681061.763
Skew:               4.484         Prob(JB):             0.000       
Kurtosis:           89.509        Condition No.:        6203501     
====================================================================
* The condition number is large (6e+06). This might indicate
strong multicollinearity or other numerical problems.
"""

Compare the percent improvement from the null model RMSE to the elastic-net and OLS model.

In [80]:
rmse_null = np.sqrt(np.mean(  (y_test - np.mean(y_train))**2  ))
rmse_null

9.403229309446852

In [81]:
rmse_elastic = np.sqrt(np.mean(  (y_test - fit_elastic_tuned.predict(x_test_std))**2  ))
print(rmse_elastic)
round((rmse_elastic - rmse_null)/rmse_null*100, 2)

9.215633926070854


-2.0

In [82]:
rmse_std_final = np.sqrt(np.mean(  (y_test - fit_std_final.predict(x_test_std_trim))**2  ))
print(rmse_std_final)
round((rmse_std_final - rmse_null)/rmse_null*100, 2)

9.215449872590813


-2.0

In [83]:
rmse_final = np.sqrt(np.mean(  (y_test - fit_final.predict(x_test_trim))**2  ))
print(rmse_final)
round((rmse_final - rmse_null)/rmse_null*100, 2)

9.215378381791666


-2.0